<a href="https://colab.research.google.com/github/truongthuanr/transfer/blob/main/08_SSSMarket_truongthuan_mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
from IPython.display import display, Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend
from tensorflow.keras.optimizers import SGD 

# Load data

In [3]:
data = np.load("/content/drive/MyDrive/01_AI_ML/sss/sss_data_full_keepna.npz")
X, y = data["arr_0"], data["arr_1"]
print(f"Loaded: {X.shape} {y.shape}")

Loaded: (4705, 280, 280, 3) (4705, 21)


In [4]:
X = tf.keras.preprocessing.image.smart_resize(X , (224,224))
print(f"Resize: {X.shape} {y.shape}")

Resize: (4705, 224, 224, 3) (4705, 21)


# Model 

In [5]:
MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/5"

In [6]:
INPUT_SHAPE = (None, 224,224,3)
OUTPUT_SHAPE = 21

In [7]:
def fbeta(y_true, y_pred, beta=1):
  # clip prediction
  y_pred = backend.clip(y_pred,0,1)
  # calculate elements
  tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
  fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
  fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
  # calculate precision
  p = tp / (tp + fp + backend.epsilon())
  # calculate recall
  r = tp / (tp + fn + backend.epsilon())
  # calculate fbeta
  bb = beta**2
  fbeta_score = backend.mean((1+bb)*(p*r)/(bb*p+r+backend.epsilon()))
  return fbeta_score

In [8]:
# Create a function which builds a Keras model
def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL):
  print("Building model with:", MODEL_URL)

  # Setup the model layers
  model = tf.keras.Sequential([
    hub.KerasLayer(MODEL_URL), # Layer 1 (input layer)
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer="he_uniform"),
    tf.keras.layers.Dense(units=OUTPUT_SHAPE, 
                          activation="sigmoid") # Layer 2 (output layer)
  ])

  # Compile the model
  model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(), # Our model wants to reduce this (how wrong its guesses are)
      optimizer=tf.keras.optimizers.Adam(), # A friend telling our model how to improve its guesses
      metrics=[fbeta] # We'd like this to go up
  )

  # Build the model
  model.build(input_shape=INPUT_SHAPE) # Let the model know what kind of inputs it'll be getting
  
  return model

In [9]:
model = create_model()

Building model with: https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/5


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1001)              5432713   
_________________________________________________________________
dense (Dense)                (None, 128)               128256    
_________________________________________________________________
dense_1 (Dense)              (None, 21)                2709      
Total params: 5,563,678
Trainable params: 130,965
Non-trainable params: 5,432,713
_________________________________________________________________


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=99)
datagen = ImageDataGenerator()
train_it = datagen.flow(X_train, y_train, batch_size=32)
test_it = datagen.flow(X_test, y_test, batch_size=32)

In [12]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
device_name

'/device:GPU:0'

In [ ]:
history = model.fit(train_it, 
                    steps_per_epoch=len(train_it), 
                    validation_data=test_it, 
                    validation_steps=len(test_it),
                    epochs=30)
# history = model.fit(X_train,y_train, 
#                     steps_per_epoch=len(train_it), 
#                     validation_data=(X_test, y_test), 
#                     validation_steps=len(test_it),
#                     epochs=30)

In [ ]:
model.save_weights('/content/drive/MyDrive/01_AI_ML/sss/model_mobilenet_catloss_adam/model_catloss_adam')

In [16]:
# Create a function which builds a Keras model
def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL):
  print("Building model with:", MODEL_URL)

  # Setup the model layers
  model = tf.keras.Sequential([
    hub.KerasLayer(MODEL_URL), # Layer 1 (input layer)
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer="he_uniform"),
    tf.keras.layers.Dense(units=OUTPUT_SHAPE, 
                          activation="sigmoid") # Layer 2 (output layer)
  ])

  # Compile the model
  opt = SGD(learning_rate=0.01, momentum=0.9)
  model.compile(
      loss='binary_crossentropy', # Our model wants to reduce this (how wrong its guesses are)
      optimizer=opt, # A friend telling our model how to improve its guesses
      metrics=[fbeta] # We'd like this to go up
  )

  # Build the model
  model.build(input_shape=INPUT_SHAPE) # Let the model know what kind of inputs it'll be getting
  
  return model

In [17]:
model = create_model()

Building model with: https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/5


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=99)
datagen = ImageDataGenerator()
train_it = datagen.flow(X_train, y_train, batch_size=32)
test_it = datagen.flow(X_test, y_test, batch_size=32)

In [19]:
history = model.fit(train_it, 
                    steps_per_epoch=len(train_it), 
                    validation_data=test_it, 
                    validation_steps=len(test_it),
                    epochs=30)

Epoch 1/30
99/99 [==============================] - 21s 174ms/step - loss: 0.3052 - fbeta: 0.2461 - val_loss: 0.2625 - val_fbeta: 0.2685
Epoch 2/30
99/99 [==============================] - 16s 166ms/step - loss: 0.2531 - fbeta: 0.2952 - val_loss: 0.2536 - val_fbeta: 0.3774
Epoch 3/30
99/99 [==============================] - 16s 166ms/step - loss: 0.2450 - fbeta: 0.3209 - val_loss: 0.2488 - val_fbeta: 0.2531
Epoch 4/30
99/99 [==============================] - 16s 166ms/step - loss: 0.2384 - fbeta: 0.3414 - val_loss: 0.2466 - val_fbeta: 0.2599
Epoch 5/30
99/99 [==============================] - 16s 166ms/step - loss: 0.2352 - fbeta: 0.3464 - val_loss: 0.2415 - val_fbeta: 0.3746
Epoch 6/30
99/99 [==============================] - 16s 165ms/step - loss: 0.2298 - fbeta: 0.3632 - val_loss: 0.2403 - val_fbeta: 0.3435
Epoch 7/30
99/99 [==============================] - 17s 167ms/step - loss: 0.2268 - fbeta: 0.3748 - val_loss: 0.2391 - val_fbeta: 0.3950
Epoch 8/30
99/99 [=======================

In [20]:
model.save_weights('/content/drive/MyDrive/01_AI_ML/sss/model_mobilenet_sgd/model_mobilenet_sgd')